# (Od)uczenie maszynowe

<img src="https://live.staticflickr.com/65535/54548177187_3b31449095_b.jpg" style="height: 450px;"/>

*Obraz wygenerowany przy użyciu modelu DALL-E.*

## Wstęp

### Motywacja

Oduczanie maszynowe (ang. *machine unlearning*) jest fascynującym i coraz bardziej popularnym tematem związanym z algorytmami uczącymi się, przede wszystkim z sieciami głębokiego uczenia. Wraz z rosnącą liczbą parametrów oraz zaawansowaniem modeli, ich zdolność do usuwania przedawnionych, błędnych lub wrażliwych informacji (np. związanych z prywatnością użytkownika) staje się kluczowa. Oduczanie umożliwia *usunięcie z pamięci modelu* wybranych informacji w sposób, który minimalizuje związaną z tym szkodę w odniesieniu do innych wyuczonych danych. Dzięki temu możemy osiągnąć bardziej precyzyjne i bezpieczne modele, bez ciągłej potrzeby trenowania modeli od początku.

### Korzyści

Wprowadzenie oduczania w świat sieci głębokiego uczenia może przynieść wiele korzyści. Pozwala między innymi na pozbycie się zdolności modelu do generowania niebezpiecznych treści. Przykładowo, jeśli generatywny model został przeszkolony na ogromnym zbiorze obrazów, ale część z nich zawiera nieodpowiednie treści, takie jak nagość, można zastosować oduczanie, aby pozbyć się konceptu nagości z pamięci modelu, co zmniejszy prawdopodobieństwo wygenerowania w przyszłości podobnych obrazów.

Po drugie, w związku z przepisami o ochronie danych osobowych (jak choćby GDPR, czyli *General Data Protection Regulation* w Unii Europejskiej), użytkownicy mają prawo wycofać zgodę na przetwarzanie ich danych przez daną firmę. Jeśli jednak model został wyuczony z użyciem tych danych, nie jest trywialne pozbycie się tej wiedzy bez konieczności ponownego treningu modelu.

Wreszcie, oduczanie może pomóc w eliminacji uprzedzeń i błędów, które mogą pojawić się w modelach, co prowadzi do bardziej sprawiedliwych i dokładnych wyników. Modele uczenia maszynowego mogą czasami przyswajać uprzedzenia i nierówności zawarte w danych treningowych (choćby związane z płcią czy kolorem skóry), co skutkuje stronniczymi predykcjami. Oduczanie pozwala na usunięcie tych uprzedzeń, co poprawia jakość i sprawiedliwość wyników.

*<u>Uwaga</u>: W klasycznym problemie oduczania maszynowego, nie tyle chcemy, żeby model nie potrafił przewidywać danej klasy, ale raczej, żeby zachowywał się tak, jakby tych danych nie było w zbiorze treningowym (co oznacza odporność modelu na atak wnioskowania o członkostwie, ang. *membership inference attack*). Stąd często pożądaną własnością nie jest bezpośrednie minimalizowanie skuteczności na pewnej części danych, tylko dążenie do tego, aby model zachowywał się wobec nich losowo lub tak jak wobec danych, których faktycznie nie było w zbiorze treningowym. Na potrzeby naszego zadania upraszczamy ten cel i chcemy, by model po prostu przewidywał daną klasę z jak najmniejszą dokładnością.*

---

## Opis problemu

Na potrzeby naszej przygody z oduczaniem będziemy analizować problem klasyfikacji na standardowym zbiorze wizji komputerowej, Fashion MNIST. Do rozwiązania tego problemu użyjemy klasycznej architektury konwolucyjnej LeNet (wizualizacja i opis poniżej). Mamy dostarczony bazowy model, który został wytrenowany na całym zbiorze danych, czyli na wszystkich dziesięciu klasach zbioru Fashion MNIST (t-shirt, spodnie, sweter, sukienka, płaszcz, sandał, koszula, but sportowy, torba, trampek). Poniżej znajduje się kod do wczytania tego modelu.

### Twoje zadanie

Twoim zadaniem jest oduczenie bazowego modelu danej wybranej klasy zbioru Fashion MNIST. Wymagamy, żeby ostatnia warstwa (warstwa klasyfikacyjna) pozostała __nienaruszona__, innymi słowy aby Twoja ingerencja była na poziomie ekstraktora cech, a nie poprzez mechaniczną modyfikację sygnału odpowiedzialnego za generowanie logitów do poszczególnych klas. Ingerencja powinna koncentrować się wyłącznie na poziomie wcześniejszych warstw modelu, z wyłączeniem jakiejkolwiek bezpośredniej zmiany w samej funkcji klasyfikującej modelu.

### Ewaluacja

Aby sprawdzić, jak sobie poradziłeś z tym zadaniem, przygotowaliśmy zestaw metryk, które pozwolą ocenić jakość Waszego rozwiązania.

Będziemy Cię oceniać pod kątem czterech aspektów:
- **Oduczenie klasyfikacji wybranej klasy** - w końcu o to chodzi w oduczaniu! Nie wiesz jednak, na oduczaniu której klasy będziesz oceniany finalnie, w ramach zbioru testowego. W ramach walidacji przyjęliśmy w tym notebooku jedną z dziesięciu klas zbioru Fashion MNIST, ale pamiętaj, że ostatecznie będziesz oceniany na innej klasie!
- **Zachowanie wysokiej wydajności na pozostałych klasach** - model musi przestać działać tylko na wybranej klasie, ale nie może stracić wydajności na pozostałych klasach - tutaj dokładność musi być jak najwyższa!
- **Ingerencja w bazowy model** - postaraj się, aby modyfikacja wag bazowego modelu była możliwie najmniejsza!
- **Różnorodność predykcji dla zapomnianej klasy** - chodzi o to, aby model był odporny na wspomniany wcześniej atak wnioskowania o członkostwie, czyli aby nie dało się poznać, że model kiedykolwiek widział dane, których ma się oduczyć!

---

### Szczegółowy opis:

#### Oduczenie i-tej klasy, $i \in \lbrace 1, ..., 10 \rbrace$.

Proces oduczenia znajomości i-tej klasy (jednej spośród dziesięciu możliwych klas) polega na __usunięciu zdolności modelu do rozpoznawania i klasyfikowania danych należących do tej konkretnej klasy__. Jest to kluczowy aspekt oduczania, który pozwala na eliminację niepożądanych lub nieodpowiednich danych z modelu.
Konkretnie, możemy mierzyć __dokładność__ (ang. accuracy) na danych __należących do konkretnej__ klasy o numerze i jako $\Psi_{i}$:

$$ \Psi_{i} = \frac{TP_i}{TP_i + FN_i}, $$

gdzie $TP_i$ oznacza liczbę przykładów poprawnie zaklasyfikowanych do klasy $i$ (odsetek prawdziwie pozytywnych), a $FN_i$ oznacza liczbę przykładów z klasy $i$, które zostały błędnie zaklasyfikowane jako należące do innej klasy (odsetek falszywie negatywnych).
W naszym problemie chcemy __minimalizować__ wartość $\Psi_{i}$.

#### Zachowanie wysokiej wydajności na pozostałych klasach 

Ważnym aspektem oduczania jest zapewnienie, że __usunięcie i-tej klasy nie wpłynie negatywnie na wydajność modelu w odniesieniu do pozostałych klas__. Model musi nadal być w stanie osiągać wysoką dokładność klasyfikacji na wszystkich pozostałych klasach. Jest to wyzwanie, ponieważ proces oduczania może  zakłócić równowagę modelu i wpłynąć negatywnie na jego zdolność do ekstrakcji cech, także w odniesieniu do pozostałych klas. Zbiór tych danych, które powinny wciąż być dobrze klasyfikowane przez model, nazywamy po prostu zbiorem pozostałych danych (ang. *remain dataset*). Możemy ocenić dokładność klasyfikacji modelu na tych danych, mierząc ją dla wszystkich klas z wyłączeniem klasy o numerze $i$ i ustalić to jako funkcję $\Phi$ zależną od $i$.

Konkretnie, możemy użyć dokładność na zbiorze pozostałym jako $\Phi_{i}$:
$$
\Phi_{i} = \frac{\sum_{k \neq i} TP_k}{\sum_{k \neq i} \left( TP_k + FN_k \right)},
$$
gdzie: $TP_k$ to liczba prawidłowo zaklasyfikowanych przykładów należących do klasy $k$, z wyłączeniem klasy $i$ (odsetek prawdziwie pozytywnych), $FN_k$ to liczba przykładów błędnie niezaklasyfikowanych jako klasa $k$, z wyłączeniem klasy $i$ (odsetek fałszywie negatywnych). Innymi słowy, licznik zawiera liczbę poprawnie zaklasyfikowanych przykładów z wszystkich klas, oprócz klasy o numerze $i$, a mianownik ilość przykładów należących do wszystkich klas, z wyłączeniem $i$-tej klasy.

#### Ingerencja w bazowy model

Proces oduczania wiąże się z ingerencją w bazowy model, co może mieć różne konsekwencje. Ważne jest, aby ta __ingerencja była minimalna__ i nie prowadziła do destabilizacji modelu. W praktyce oznacza to, że zmiany wprowadzone w modelu powinny być ograniczone do niezbędnych modyfikacji, które pozwolą na usunięcie klasy i-tej, bez wpływu na strukturę i funkcjonalność modelu, zatem z minimalną modyfikacją parametrów modelu. Wymagamy, aby __ostatnia warstwa w modelu LeNet została nienaruszona__, ale będziemy też mierzyć odległość między wyjściowym a oduczonym modelem, zaproponowanym przez Ciebie. Jest to również sposób na lepsze zrozumienie działania modelu. Jeśli odległość od bazowego modelu nie jest zbyt duża, a mamy zaufanie do bazowego modelu, to uzyskany przez Ciebie model jest wystarczająco bliski, aby wzbudzać nasze zaufanie. 

W naszym problemie będziemy posługiwać się tradycyjną **odległością** $\mathbf{L_2}$, którą można wyrazić wzorem:

$$ \mathcal{d}_{dist} = \mathcal{L_2} (\theta; \theta_0) = \sqrt{\sum_j (\theta_j - \theta_{0,j})^2}, $$

gdzie
- $\theta$ oznacza wektor parametrów modelu po procesie oduczania,
- $\theta_0$ oznacza wektor parametrów bazowego modelu,
- $\theta_j$ oraz $\theta_{0,j}$ to odpowiednie wartości parametrów dla j-tego warstwy modeli.

Odległość $\mathcal{L}_2$ sumuje kwadraty różnic odległości między kolejnymi warstwami modelu bazowego w odniesieniu do odpowiadających warstw zmodyfikowanego modelu, a finalnie liczy pierwiastek z tej sumy. Im mniejsza wartość $\mathcal{d}_{dist}$, tym mniejsza ingerencja w bazowy model, co jest pożądane w kontekście minimalizacji zmian w strukturze modelu.

#### Różnorodność predykcji dla zapomnianej klasy

Ostatnim aspektem oceny jest __różnorodność predykcji dla zapomnianej klasy__. Po procesie oduczania model powinien generować różnorodne predykcje dla danych, które należą do klasy $i$, którą miał zapomnieć model. Oznacza to, że model powinien po prostu przypisywać je do innych klas w sposób zróżnicowany tak, aby nie dało się łatwo stwierdzić, czy dane te znajdowały się pierwotnie w zbiorze treningowym bazowego modelu.

Model nie powinien również przypisywać wszystkich predykcji do jednej konkretnej klasy, ponieważ mogłoby to być interpretowane jako scalenie wybranych klas, co nie jest przez nas pożądane.

Aby zmierzyć różnorodność predykcji, wykorzystamy __dywergencję Kullbacka-Leiblera (KL)__. Jest to standardowa miara rozbieżności między dwoma rozkładami prawdopodobieństwa. W naszym przypadku badamy odległość rozkładu predykcji modelu od rozkładu jednostajnego, co oznacza, że żadna z pozostałych klas nie powinna być faworyzowana.

Dywergencję KL definiujemy jako:

$$
\mathcal{D}_{KL}(p \parallel u) = \sum_{c=1}^{C} p(c) \log \left( \frac{p(c)}{u(c)} \right),
$$

gdzie:
- $p(c)$ to prawdopodobieństwo przypisane przez model próbki do klasy $c$,
- $u(c)$ to prawdopodobieństwo zgodne z rozkładem jednostajnym, czyli $u(c) = \frac{1}{C}$,
- $C$ to liczba wszystkich klas.

W przypadku idealnej różnorodności predykcji dla zapomnianej klasy, rozkład $p(c)$ powinien być jak najbardziej zbliżony do rozkładu jednostajnego $u(c)$, co oznacza minimalną wartość $\mathcal{D}_{KL}$.

Jeśli KL dywergencja jest wciąż dość skomplikowana, nie martw się - w zakładce **Informacje Uzupełniające** dostarczyliśmy kilka pomocnych wskazówek. Dodatkowo zerknij na kod definicji funkcji - pozostawiliśmy w nim kilka komentarzy.

---

Wszystkie z powyższych czterech celów będziemy ewaluować na danych Fashion MNIST, **jednak nie wiesz, której z klas będziemy chcieli się finalnie oduczyć**. **Ponadto nie wiesz, jaki podzbiór danych z poszczególnych klas będzie wykorzystany do oduczania.** Do przygotowania rozwiązania możesz wybrać sobie dowolną klasę, ale rozwiązanie to musi być gotowe do oduczenia każdej z klas tego zbioru. O ile rozkład danych w zbiorze testowym będzie podobny do rozkładu w tym notebooku, postaraj się uniknąć nadmiernego dopasowania do wybranych danych.

Sprawdź więc, czy Twoje rozwiązanie jest uniwersalne. Upewnij się, że działa na różnych etykietach zleconych do zapomnienia oraz różnych zbiorach danych (architektura jest niezmienna, dopasowana do przetwarzania danych wizyjnych o wymiarze $28 \times 28$).

#### Finalna ocena

Całościowo, Twój wynik można matematycznie zapisać jako suma ważona:
$$
\Sigma_{score} = \frac{1}{4} \cdot \Sigma_{target} + \frac{1}{4} \cdot \Sigma_{remain} + \frac{1}{4} \cdot \Sigma_{dist} + \frac{1}{4} \cdot \Sigma_{kl}
$$

*<u>Uwaga</u>: Metrykę $\Phi$ chcielibyśmy maksymalizować, a pozostałe, czyli $\Psi$, $d_{dist}$ oraz $\mathcal{D}_{KL}$, minimalizować. Stąd należy zwrócić uwagę kolejność odejmowania przy liczeniu poszczególnych składników $\Sigma$ do końcowej ewaluacji!*

$\Sigma_{target}$ – punktacja za skuteczność oduczania na wybranej klasie (dla uproszczenia, w zadaniu przyjmujemy, że im niższa skuteczność na klasie docelowej, tym lepiej; skalowana w zakresie [0, 100] według progów $[0.09, 0.3]$):

$$
\Sigma_{target}(\Psi_{i}) = 
\begin{cases}
100 & \text{jeśli } \Psi_{i} \leq 0.09 \\
0 & \text{jeśli } \Psi_{i} \geq 0.3 \\
100 \cdot \dfrac{0.3 - \Psi_{i}}{0.3 - 0.09} & \text{w przeciwnym razie}
\end{cases}
$$

$\Sigma_{other}$ – punktacja za zachowanie dokładności na pozostałych klasach (im wyższa skuteczność, tym wyższy wynik tej metryki; skalowana w zakresie [0, 100] według progów $[0.87, 0.90]$):

$$
\Sigma_{other}(\Phi_{i}) = 
\begin{cases}
100 & \text{jeśli } \Phi_{i} \geq 0.90 \\
0 & \text{jeśli } \Phi_{i} \leq 0.87 \\
100 \cdot \dfrac{\Phi_{i} - 0.87}{0.90 - 0.87} & \text{w przeciwnym razie}
\end{cases}
$$

$\Sigma_{dist}$ – punktacja za stopień ingerencji w model (im mniejsza odległość $L_2$, tym wyższy wynik tej metryki; skalowana w zakresie [0, 100] według progów $[1.3, 3.0]$):

$$
\Sigma_{dist}(\mathcal{d}_{dist}) = 
\begin{cases}
100 & \text{jeśli } \mathcal{d}_{dist} \leq 1.3 \\
0 & \text{jeśli } \mathcal{d}_{dist} \geq 3.0 \\
100 \cdot \dfrac{3.0 - \mathcal{d}_{dist}}{3.0 - 1.3} & \text{w przeciwnym razie}
\end{cases}
$$

$\Sigma_{kl}$ – punktacja za różnorodność predykcji (im niższa dywergencja KL, tym wyższy wynik tej metryki; skalowana w zakresie [0, 100] według progów $[0.2, 0.5]$):

$$
\Sigma_{kl}(\mathcal{D}_{KL}) = 
\begin{cases}
100 & \text{jeśli } \mathcal{D}_{KL} \leq 0.2 \\
0 & \text{jeśli } \mathcal{D}_{KL} \geq 0.5 \\
100 \cdot \dfrac{0.5 - \mathcal{D}_{KL}}{0.5 - 0.2} & \text{w przeciwnym razie}
\end{cases}
$$

**Jednakże**, zadanie jest oceniane **z góry na 0 punktów we wszystkich kategoriach**, jeżeli Twoje rozwiązanie:
- nie zastosuje się do wytycznych, czyli przykładowo:
    - będzie wprowadzona zmiana w architekturze sieci; 
    - zostanie zmodyfikowana ostatnia warstwa sieci;
    - dokonana będzie jakakolwiek próba oszustwa, polegająca np. na modyfikacji funkcji ewaluacyjnej;
- rozwiązanie będzie niesatysfakcjonujące:
    - dokładność klasyfikacji na zbiorze klasy do zapomnienia utrzyma się powyżej 0.5;
    - dokładność klasyfikacji na zbiorze pozostałych klas spadnie poniżej 0.75;
    - dystans $L_2$ będzie większy niż 8.0;
    - wartość dywergencji Kullbacka-Leiblera przekroczy 1.75.

Ponadto, zaproponowane przez Ciebie oduczanie modelu, tj. działanie funkcji *unlearn()*, może trwać nie dłużej niż 5 minut z użyciem GPU.

Za zadanie możesz otrzymać maksymalnie 100 punktów.

Pamiętaj, że podczas sprawdzania flaga *FINAL_EVALUATION_MODE* zostanie ustawiona na True.

Powodzenia!

---

## Kod startowy

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# W razie potrzeby, importuj dodatkowe biblioteki poniżej, u siebie w kodzie.

import os
from copy import deepcopy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tarfile
import tempfile

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Używam urządzenia: {DEVICE}")

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

FINAL_EVALUATION_MODE = False  # Podczas sprawdzania ustawimy tę flagę na True.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

seed = 101

os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

### Dane

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class FilteredFashionMNIST(datasets.FashionMNIST):
    def __init__(self, *args, classes=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.classes = classes
        if self.classes is not None:
            self.data, self.targets = self._filter_classes(
                self.data, self.targets, self.classes)

    def _filter_classes(self, data, targets, classes):
        mask = torch.zeros_like(targets, dtype=torch.bool)
        for c in classes:
            mask = mask | (targets == c)
        return data[mask], targets[mask]

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

BATCH_SIZE = 64

tempdir = tempfile.TemporaryDirectory()
TMP_DIR = tempdir.name
DATA_PATH = os.path.join(TMP_DIR, "data")

if not FINAL_EVALUATION_MODE:
    import gdown

    GDRIVE_DATA = [
        ("1SeXrzvs64MBG57ayK6965cya3WRfCMx_", "data/FashionMNIST.tar.gz"),
        ("1mqzxg-_0PJjvErvfwOGad6siNcQKqwk5", "data/FilteredFashionMNIST.tar.gz"),
        ("1YSn8EFjbYDcDCVdlByA_kDKCg4VZLwH8", "models/lenet_base_final.pt"),
    ]
    
    for file_id, output in GDRIVE_DATA:        
        url = f'https://drive.google.com/uc?id={file_id}'
        os.makedirs(os.path.dirname(output), exist_ok=True)
        gdown.download(url, output, quiet=False)
        print(f"Downloaded: {output}")

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def unpack_tar_gz(filename: str, path: str = DATA_PATH) -> None:
    """Rozpakowuje archiwum tar.gz do wskazanego katalogu."""
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall(path=path)

unpack_tar_gz("data/FashionMNIST.tar.gz", os.path.join(DATA_PATH, "FashionMNIST"))
unpack_tar_gz("data/FilteredFashionMNIST.tar.gz", os.path.join(DATA_PATH, "FilteredFashionMNIST"))

transform_fashion = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.2860,), (0.3530,))
    ])

def get_data_dict():
    def create_filtered_loader(classes):
        dataset = FilteredFashionMNIST(
            root=DATA_PATH,
            download=True,
            transform=transform_fashion,
            classes=classes)
        loader = DataLoader(
            dataset=dataset,
            batch_size=BATCH_SIZE,
            shuffle=True)
        return loader

    loader_fashion = DataLoader(
        dataset=datasets.FashionMNIST(
            root=DATA_PATH,
            download=True,
            transform=transform_fashion),
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    class_groups = {}
    num_classes = 10
    for i in range(num_classes):
        class_groups[f"{i}"] = [i]
        class_groups[f"~{i}"] = [j for j in range(num_classes) if j != i]

    data_dict = {
        "fashion": {
            "loader": loader_fashion,
        }
    }

    for group_name, classes in class_groups.items():
        loader = create_filtered_loader(classes)
        data_dict["fashion"][f"loader_{group_name}"] = loader
    return data_dict

data_dict = get_data_dict()

### Model

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        latent_dim = 256
        self.fc = nn.Linear(latent_dim, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

<img src="https://live.staticflickr.com/65535/54549285793_e078de98d3_b.jpg" style="height: 450px;"/>

Źródło: GeeksForGeeks

### Ładowanie modelu

In [ ]:
pretrained_model = LeNet(num_classes=10)
initial_state_dict = torch.load("./models/lenet_base_final.pt")
pretrained_model.load_state_dict(initial_state_dict)
pretrained_model = pretrained_model.to(DEVICE)

### Metryki

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def evaluate_target(model, data_loader, criterion, device="cpu"):
    """
    Funkcja ocenia wydajność modelu na danych docelowych.
    
    :param model: Model PyTorch, który chcemy ocenić.
    :param data_loader: DataLoader zawierający dane do oceny dokładności klasyfikacji.
    :param criterion: Funkcja kosztu używana do obliczenia straty.
    :param device: Urządzenie, na którym wykonywane są obliczenia (domyślnie "cpu").
    :return: Krotka zawierająca średnią stratę oraz dokładność na danych docelowych.
    """
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return total_loss / len(data_loader.dataset), correct / len(data_loader.dataset)

def measure_target_uniformity(model, loader_target, device="cpu", num_classes=10):
    """
    Mierzy, jak bliskie są predykcje modelu dla klasy docelowej do rozkładu jednostajnego.
    Używamy dywergencji KL: KL(U || p(x)) lub KL(p(x) || U) jako miary jednostajności.

    Obliczamy średnią KL(p || U) = sum_{y} p(y) log [p(y) / (1/num_classes)].
    Niższa wartość KL oznacza większą jednostajność.
    Dokładne wytłumaczenie poniżej w informacjach uzupełniających.
    """
    model.eval()
    kl_sum = 0.0
    total_samples = 0

    with torch.no_grad():
        for images, _ in loader_target:
            images = images.to(device)
            logits = model(images)
            log_probs = F.log_softmax(logits, dim=1)
            probs = torch.exp(log_probs)
            entropy_term = (probs * log_probs).sum(dim=1)
            kl_batch = entropy_term + torch.log(
                torch.tensor(num_classes, device=device)
            )
            kl_sum += kl_batch.sum().item()
            total_samples += images.size(0)

    return kl_sum / total_samples
    
def measure_l2_distance(model, initial_state_dict, device="cpu"):
    """
    Mierzy odległość L_2 pomiędzy aktualnym stanem modelu a jego początkowym stanem.
    """
    l2_distance = 0.0
    for name, param in model.named_parameters():
        if param.requires_grad:
            initial_param = initial_state_dict[name]
            l2_distance += torch.sum((param.to(device) - initial_param.to(device)) ** 2).item()
    return l2_distance ** 0.5

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

WEIGHTS = [0.25, 0.25, 0.25, 0.25]

acc_forget_thresholds = [0.09, 0.3]
acc_other_thresholds = [0.87, 0.90]
dist_thresholds = [1.3, 3.0]
d_kl_thresholds = [0.2, 0.5]

acc_forget_absolute_threshold = 0.5
acc_target_absolute_threshold = 0.75
distance_absolute_threshold = 8.0
dkl_absolute_threshold = 1.75

def compute_final_score(
        kl_after, acc_rest_after, l2_dist, acc_target_after
    ):
    """
    Funkcja oblicza końcowy wynik na podstawie różnych metryk.
    :param kl_after: Wartość KL po treningu.
    :param acc_rest_after: Dokładność na danych ze zbioru, który model ma pamiętać.
    :param l2_dist: Odległość L2 po treningu.
    :param acc_target_after: Dokładność na danych ze zbioru, który model ma zapomnieć.
    :return: Końcowy wynik."""

    def compute_metric_score(value, thresholds, maximize=True):
        if maximize:
            if value >= thresholds[1]:
                return 100
            elif value <= thresholds[0]:
                return 0
            else:
                return 100 * (value - thresholds[0]) / (thresholds[1] - thresholds[0])
        else:
            if value <= thresholds[0]:
                return 100
            elif value >= thresholds[1]:
                return 0
            else:
                return 100 * (thresholds[1] - value) / (thresholds[1] - thresholds[0])
    print(f"Dywergencja KL: {kl_after:.2f}")
    print(f"Dokładność klasyfikacji na pozostałym zbiorze: {acc_rest_after:.2f}")
    print(f"Odległość L2: {l2_dist:.2f}")
    print(f"Dokładność na zbiorze do zapomnienia: {acc_target_after:.2f}")

    if acc_rest_after < acc_target_absolute_threshold or acc_target_after > acc_forget_absolute_threshold \
            or l2_dist > distance_absolute_threshold or kl_after > dkl_absolute_threshold:
        return 0

    kl_score = compute_metric_score(kl_after, d_kl_thresholds, maximize=False)
    acc_rest_score = compute_metric_score(acc_rest_after, acc_other_thresholds, maximize=True)
    l2_dist_score = compute_metric_score(l2_dist, dist_thresholds, maximize=False)
    acc_target_score = compute_metric_score(acc_target_after, acc_forget_thresholds, maximize=False)

    total_score = (
        WEIGHTS[0] * kl_score +
        WEIGHTS[1] * acc_rest_score +
        WEIGHTS[2] * l2_dist_score +
        WEIGHTS[3] * acc_target_score
    )

    return total_score
    
def evaluate_model(model, data_loader_target, data_loader_rest, initial_state_dict, device):
    """
    Funkcja ocenia model na danych docelowych i pozostałych, sprawdza odległość L2
    oraz podobieństwo do rozkładu jednostajnego.
    :param model: model do oceny.
    :param data_loader_target: DataLoader dla danych docelowych.
    :param data_loader_rest: DataLoader dla pozostałych danych.
    :param initial_state_dict: początkowy stan modelu.
    :param criterion: funkcja kosztu.
    :param device: urządzenie do obliczeń.
    :return: ocena modelu.
    """
    model.eval()
    criterion = nn.CrossEntropyLoss()
    _, acc_rest = evaluate_target(model, data_loader_rest, criterion, device)
    _, acc_target = evaluate_target(model, data_loader_target, criterion, device)
    l2_dist = measure_l2_distance(model, initial_state_dict, device)
    kl = measure_target_uniformity(model, data_loader_target, device, num_classes=10)
    return compute_final_score(kl, acc_rest, l2_dist, acc_target)

### Zgodność

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def has_same_last_layer(model1: LeNet, model2: LeNet) -> bool:
    return all(torch.equal(p1, p2) for p1, p2 in zip(model1.fc2.parameters(), model2.fc2.parameters()))

def has_same_architecture(model: LeNet) -> bool:
    return set([k for (k, v) in list(model.named_parameters())]) == \
        {'block1.0.weight', 'block1.0.bias', 'block1.1.weight', 'block1.1.bias', \
         'block2.0.weight', 'block2.0.bias', 'block2.1.weight', 'block2.1.bias', \
         'fc.weight', 'fc.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'}

### Trywialne rozwiązanie

In [ ]:
def dumb_solution(model, data, target_class):
    """
    Funkcja, która dodaje szum do wag modelu, aby zmienić jego zachowanie na danych docelowych.
    :param model: Model PyTorch, który chcemy zmodyfikować.
    :param data: Dane, na których chcemy zmodyfikować model.
    :param target_class: Klasa docelowa, na której chcemy zmodyfikować model.
    """
    new_model = deepcopy(model)  # Tworzymy kopię modelu, aby nie modyfikować oryginału
    with torch.no_grad():
        for name, param in new_model.named_parameters():
            if "fc2" not in name:  # Ostatnia wartstwa pozostaje niezmieniona
                param.add_(torch.randn_like(param) * 0.1)  # Dodawanie szumu do wag
    return new_model

#### Trywialne rozwiązanie - ewaluacja

In [ ]:
if not FINAL_EVALUATION_MODE:
    perturbed_model = dumb_solution(pretrained_model, None, None)
    assert has_same_last_layer(pretrained_model, perturbed_model), "Ostatnia warstwa modelu nie jest taka sama jak w oryginalnym modelu."
    assert has_same_architecture(perturbed_model), "Architektura modelu nie jest taka sama jak w oryginalnym modelu."

    target_class = 9
    print(f"Klasa docelowa: {target_class}")
    target_loader = data_dict["fashion"][f"loader_{target_class}"]
    other_loader = data_dict["fashion"][f"loader_~{target_class}"]

    score = evaluate_model(perturbed_model, target_loader, other_loader, initial_state_dict, DEVICE)
    print(f"Wynik: {score:.2f}")

## Informacje uzupełniające

#### Wyjaśnienie dywergencji Kullbacka-Leiblera

Dywergencja Kullacka-Leiblera (ang. *Kullback-Leibler divergence*), KL, to miara rozbieżności między dwoma rozkładami prawdopodobieństwa. W kontekście uczenia maszynowego i statystyki, Dywergencja KL pozwala ocenić, jak bardzo jeden rozkład różni się od drugiego. Formalnie, dla dwóch dyskretnych rozkładów $P$ i $Q$, jest ona zdefiniowana jako:

$$
D_{KL}(P \parallel Q) = \sum_{x \in X} P(x) \log \frac{P(x)}{Q(x)}
$$

Gdzie:
- $P(x)$ to prawdziwy rozkład (np. dane rzeczywiste),
- $Q(x)$ to aproksymowany rozkład (np. predykcja modelu),
- $X$ to przestrzeń zdarzeń.

Dywergencja KL nie jest miarą symetryczną, co oznacza, że $D_{KL}(P \parallel Q) \neq D_{KL}(Q \parallel P)$. Wartość dywergencji wynosi 0, gdy oba rozkłady są identyczne.


##### Przykłady porównań rozkładów dyskretnych:

1. **Przykład 1: Identyczne rozkłady**
    - $P = [0.4, 0.6]$
    - $Q = [0.4, 0.6]$
    - $D_{KL}(P \parallel Q) = 0$

2. **Przykład 2: Rozkłady różniące się nieznacznie**
    - $P = [0.4, 0.6]$
    - $Q = [0.5, 0.5]$
    - $D_{KL}(P \parallel Q) \approx 0.02$

3. **Przykład 3: Rozkłady bardzo różne**
    - $P = [0.9, 0.1]$
    - $Q = [0.1, 0.9]$
    - $D_{KL}(P \parallel Q) \approx 0.75$.

##### Wizualizacja KL Dywergencji

Poniżej znajduje się kod w Pythonie, który wizualizuje KL dywergencję dla dwóch dyskretnych rozkładów.

```python
import numpy as np
import matplotlib.pyplot as plt

def kl_divergence(p, q):
     """Oblicza KL dywergencję między dwoma rozkładami."""
     p = np.array(p)
     q = np.array(q)
     return np.sum(p * np.log(p / q))

# Przykładowe rozkłady
P = [0.4, 0.6]
Q_list = [
     [0.4, 0.6],  # Identyczny rozkład
     [0.5, 0.5],  # Nieznacznie różny
     [0.9, 0.1]   # Bardzo różny
]

# Obliczanie KL dywergencji
kl_values = [kl_divergence(P, Q) for Q in Q_list]

# Wizualizacja
labels = ['Q1 (identyczny)', 'Q2 (nieznacznie różny)', 'Q3 (bardzo różny)']
x = np.arange(len(Q_list))

plt.bar(x, kl_values, color='skyblue')
plt.xticks(x, labels, rotation=15)
plt.ylabel('KL Dywergencja')
plt.title('KL Dywergencja dla różnych rozkładów Q względem P')
plt.show()
```


##### Interpretacja

- Gdy $P$ i $Q$ są identyczne, KL dywergencja wynosi 0.
- Gdy $Q$ różni się od $P$, KL dywergencja rośnie.
- KL dywergencja nie jest symetryczna, więc zmiana kolejności $P$ i $Q$ zmienia wynik. Warto o tym pamiętać przy interpretacji wyników.
- Możesz dostrzec różnicę między wzorem z tego zadania, a wzorem z funkcji *measure_target_uniformity()*. Wynika to z faktu, że zestawiamy nasz rozkład prawdopodobieństwa z rozkładem jednostajnym, a wówczas wzór przekształca się do tamtej postaci.

## Pliki zgłoszeniowe

Od Ciebie będziemy potrzebować zaledwie tego notebooka - wraz z kluczową definicją metodą *unlearn*, która pozwoli znaleźć ostateczne wagi modelu, na których będzie wykonana ewaluacja na wybranej przez nas klasie ze zbioru Fashion MNIST.

## Ograniczenia

Zaproponowaliśmy takie wytyczne, które wymuszają niską skuteczność modelu na klasie docelowej. Pamiętaj, że w ogólnym przypadku celem naszego algorytmu oduczania powinno być osiągnięcie stanu, w którym model zachowuje się tak, jakby dane z klasy docelowej nigdy nie były częścią zbioru treningowego.

Dodatkowo zakazujemy modyfikacji hiperparametrów architektury, a także jakichkolwiek parametrów warstwy klasyfikującej.

Oczekujemy, aby Twoje rozwiązanie opierało się i wykorzystywało jedynie standardowe biblioteki używane w uczeniu maszynowym, takie jak torch, numpy, matplotlib/seaborn, scikit-learn. 

# Twoje rozwiązanie

In [ ]:
def unlearn(model, data, target_class):
    """
    Funkcja, która wykonuje operację "unlearn" na modelu.
    :param model: Model, który chcemy zmodyfikować.
    :param data: Dane, na których chcemy zmodyfikować model, zawierające
                 pełen zbiór danych.
    :param target_class: Indeks klasy docelowej, którą chcemy oduczyć model.
    return: Zmodyfikowany model.
    """
    # tutaj zaproponuj swoje rozwiązanie
    # ... 
    return model

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    def prepare_data(data_dict, target_class):
        """
        Przygotowuje dane do treningu modelu.
        :param data_dict: Słownik z danymi.
        :param target_class: Klasa docelowa.
        :return: Krotka z danymi do zapomnienia i pozostałymi danymi.
        """
        target_data = data_dict["fashion"][f"loader_{target_class}"]
        other_data = data_dict["fashion"][f"loader_~{target_class}"]
        return target_data, other_data
    
    target_class = 9
    unlearned_model = unlearn(pretrained_model, data_dict, target_class)
    target_loader, other_loader = prepare_data(data_dict, target_class)  
    score = evaluate_model(unlearned_model, target_loader, other_loader, initial_state_dict, DEVICE)
    print(f"Ocena modelu po oduczaniu: {score}")

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if FINAL_EVALUATION_MODE:
    import cloudpickle

    OUTPUT_PATH = "file_output"
    FUNCTION_FILENAME = "your_model.pkl"
    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, FUNCTION_FILENAME)

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(unlearn, f)